# Start Simulation

In [1]:
# env.clean(clean_all=True)

In [2]:
import random
import numpy as np
import os
import time
from PIL import Image

import torch
torch.__version__
device = torch.device("cuda")

In [3]:
import getpass
user = getpass.getuser()
print(user)

yizhou


In [4]:
usd_path = f"omniverse://localhost/Users/{user}/UVA/uva_table.usd"

In [5]:
from omni.isaac.kit import SimulationApp    
simulation_app = SimulationApp({"headless": True, "open_usd": usd_path,  "livesync_usd": usd_path}) 

Starting kit application with the fillowing args:  ['/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps', '--/physics/cudaDevice=0', '--portable', '--no-window']
Passing the following args to the base kit application:  ['-f', '/home/yizhou/.local/share/jupyter/runtime/kernel-097307b0-7a63-

OmniAssetFileFormat


Warp initialized:
   Version: 0.2.2
   CUDA device: NVIDIA GeForce RTX 3090
   Kernel cache: /home/yizhou/.cache/warp/0.2.2
[2.105s] [ext: omni.kit.menu.create-1.0.2] startup
[2.108s] [ext: omni.mdl-0.1.0] startup
[2.129s] [ext: omni.kit.window.file_importer-1.0.4] startup
[2.131s] [ext: omni.kit.window.file_exporter-1.0.4] startup
[2.132s] [ext: omni.kit.material.library-1.3.10] startup
[2.136s] [ext: omni.kit.window.drop_support-1.0.0] startup
[2.137s] [ext: omni.kit.window.file-1.3.16] startup
[2.140s] [ext: omni.kit.context_menu-1.3.9] startup
[2.144s] [ext: omni.kit.window.property-1.6.3] startup
[2.145s] [ext: omni.kit.window.content_browser-2.4.28] startup
[2.157s] [ext: omni.kit.widget.stage-2.6.15] startup
[2.164s] [ext: omni.isaac.version-1.0.0] startup
[2.165s] [ext: omni.kit.property.usd-3.14.9] startup
[2.251s] [ext: omni.kit.viewport.legacy_gizmos-1.0.0] startup
[2.255s] [ext: omni.hydra.rtx-0.1.0] startup
[2.268s] [ext: omni.renderer-rtx-0.0.0] startup
[2.270s] [ext: omn

In [6]:
# set log level
import logging
import carb

logging.getLogger("omni.hydra").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.urdf").setLevel(logging.ERROR)
logging.getLogger("omni.physx.plugin").setLevel(logging.ERROR)

logging.getLogger("omni.isaac.synthetic_utils").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.synthetic_utils.syntheticdata").setLevel(logging.ERROR)
logging.getLogger("omni.hydra.scene_delegate.plugin").setLevel(logging.ERROR)


l = carb.logging.LEVEL_ERROR
carb.settings.get_settings().set("/log/level", l)
carb.settings.get_settings().set("/log/fileLogLevel", l)
carb.settings.get_settings().set("/log/outputStreamLevel", l)

# # This logged error is printed as it should
# carb.log_error("ERROR")
# # This warning is printed but should not
# carb.log_warn("WARNING")

# Config

In [7]:
from task.config import DATA_PATH, FEATURE_PATH
task_type = "Table"
side_choice = "Border"
base_asset_id = 0
load_nucleus = True

In [8]:
pause

NameError: name 'pause' is not defined

# Init Env

In [9]:
from uva_env import UvaEnv
env = UvaEnv()

In [10]:
from task.utils import add_scene_default
add_scene_default()

In [13]:
print(list(env.stage.TraverseAll()))

[Usd.Prim(</physicsScene>), Usd.Prim(</World>), Usd.Prim(</World/defaultLight>), Usd.Prim(</World/physicsScene>), Usd.Prim(</OmniverseKit_Persp>), Usd.Prim(</OmniverseKit_Front>), Usd.Prim(</OmniverseKit_Top>), Usd.Prim(</OmniverseKit_Right>), Usd.Prim(</Render>), Usd.Prim(</Render/RenderProduct_Viewport>), Usd.Prim(</Render/Vars>), Usd.Prim(</Render/Vars/LdrColor>)]


In [12]:
env.clean()
env.world.step(render=True)

# Scene

In [14]:
from task.scene import ArrangeScene
scene = ArrangeScene(task_type, side_choice, base_asset_id = 0, traj_id = 0, load_nucleus = load_nucleus)
env.scene = scene

In [15]:
# add base
scene.add_base_asset()

asset_prim: omniverse://localhost/Users/yizhou/Asset/T/Appleseed/Appleseed_CoffeeTable.usd


In [16]:
# add room
scene.add_room()

asset_prim: omniverse://localhost/Users/yizhou/Asset/S/0/layout.usd


In [17]:
env.world.step(render=True)

In [18]:
# randomize scene
from layout.randomizer import Randomizer
randomizer = Randomizer(load_nucleus = load_nucleus)

# Reward

In [19]:
from uv.reward import Rewarder

In [20]:
rewarder = Rewarder(env.world)
env.rewarder = rewarder

# Render

In [21]:
from render.helper import RenderHelper
render = RenderHelper(task_type, side_choice)

render.add_task_cameras(camera_type="main")
render.add_task_cameras(camera_type="side")
render.set_cameras()

In [22]:
render.camera_paths

['/World/render/camera_main', '/World/render/camera_side']

In [23]:
pause

NameError: name 'pause' is not defined

#  Learning

In [24]:
# from learning.network.resnet import ResNetFeatureExtractor

# feature extraction
from learning.utils import extract_image_clip_feature_and_save, obtain_action_from_trainer
from transformers import CLIPProcessor, CLIPModel

feature_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
feature_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# replay buffer
import json
from learning.replay_buffer import ReplayBuffer
from learning.config import *

buffer = ReplayBuffer(max_size=2000)

# trainer
# from learning.network.sac import *

# policy = Policy()

# qf1 = QFunction()
# qf2 = QFunction()
# target_qf1 = QFunction()
# target_qf2 = QFunction()

# from learning.sac_trainer import SACTrainer
# from learning.ddpg_trainer import DDPGTrainer

# trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
#      use_automatic_entropy_tuning = False, 
#      policy_lr=1e-3, 
#      qf_lr=1e-3,
#      target_update_period = 5)

In [25]:
model_type = "ddpg"

if model_type == "sac":
    # trainer
    from learning.network.sac import *

    policy = Policy()
    target_policy = Policy()

    qf1 = QFunction()
    qf2 = QFunction()

    target_qf1 = QFunction()
    target_qf1.eval()

    target_qf2 = QFunction()
    target_qf2.eval()

    from learning.sac_trainer import SACTrainer

    trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
         use_automatic_entropy_tuning = True, 
         policy_lr=1e-3, 
         qf_lr=1e-3,
         target_update_period = 5)

else: # ddpg
    # trainer
    from learning.network.ddpg import *

    policy = Policy()
    target_policy = Policy()

    qf1 = QFunction()
    target_qf1 = QFunction()
    target_qf1.eval()

    from learning.ddpg_trainer import DDPGTrainer
    
    trainer = DDPGTrainer(qf1, target_qf1, policy, target_policy,
     policy_learning_rate=1e-3,
     qf_learning_rate=1e-3
    )

# Trajectory

In [26]:
use_network = True
debug = True

In [27]:
total_traj = 0
total_step = 0

---------------------

In [28]:
# traj config
for traj_id in range(0, 100):
    print("total_traj: ", total_traj)
    env.scene.traj_id = traj_id
    
    # base
    if total_traj % 5 == 0:
        asset_count = len(env.scene.base_asset_file_paths)
        base_asset_id = random.choice([_ for _ in range(asset_count)])
        env.scene.base_asset_id = base_asset_id    
        env.clean(clean_base = True, clean_all = False)
        scene.add_base_asset()
        
        # randomize scene
        randomizer.randomize_house(rand = True)
        
    image_folder = os.path.join(DATA_PATH, task_type, side_choice, str(traj_id))
    
    env.world.step(render = True)
    env.world.step(render = True)

    # get images
    env.world.render()
    images = render.get_images()
    render.save_rgb(images[0]['rgb'], image_folder, "0")
    render.save_rgb(images[1]['rgb'], image_folder, "0_side")
    
    ## extract feature
    
#     extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#         feature_extractor, os.path.join(image_folder, str(0) + ".pt"))
    if use_network:
        extract_image_clip_feature_and_save(f"{image_folder}/{0}.png", feature_model, feature_processor, 
            f"{image_folder}/{0}.pt",)
    

    # trajectory
    for step in range(5):
        total_step += 1
        
        # sample an object
        env.add_scene_obj(mode = "random")
        
        # TODO: get action from sampling
        if not use_network or total_traj < 10 or np.random.rand() < 0.2:
            x, y = np.tanh(np.random.randn()), np.tanh(np.random.randn())
        else:
            image_feature_file = f"{image_folder}/{step}.pt"
            
            object_info = env.scene.objects[-1]
            object_type = object_info["type"]
            obj_name = object_info["name"][:-4]
            object_feature_file = os.path.join(FEATURE_PATH, object_type, obj_name + ".pt")
            x, y = obtain_action_from_trainer(image_feature_file, object_feature_file, trainer, 
                                              scaler=np.exp(- total_traj / 100))
            object_info["use_network"] = True
        
        # load the object into the scene
        env.put_last_object((x, y)) 
        env.world.step(render=True)
        
        # register the object to the world for physics update
        env.register_last_object()
        env.world.step(render=True)

        # get images
        env.world.render()
        images = render.get_images()
        render.save_rgb(images[0]['rgb'], image_folder, str(step + 1))
        render.save_rgb(images[1]['rgb'], image_folder, f"{step + 1}_side")

        ## calculate reward
        env.calculate_last_reward(simulation_step = 30)
        
        ## extract feature
#         extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#             feature_extractor, os.path.join(image_folder, str(step + 1) + ".pt"))
        if use_network:
            extract_image_clip_feature_and_save(f"{image_folder}/{step + 1}.png", feature_model, feature_processor, 
                f"{image_folder}/{step + 1}.pt",)
    

        ## reset
        env.world.reset()
        env.world.step(render=True)
        
        ## trainer nework
        if use_network and total_step % UPDATE_TRAINER_STEPS == 0 and total_traj > 5:
            batch = buffer.sample_batch(batch_size = BATCH_SIZE)
            trainer.update(batch)
            
            if debug and total_step % 10 == 0:
                rewards = batch['rewards'].to(device)
                terminals = batch['terminals'].to(device)
                obs = batch['observations'].to(device)
                actions = batch['actions'].to(device)
                next_obs = batch['next_observations'].to(device)
                obj_features = batch['object_features'].to(device)
                
                dist = trainer.policy(obs, obj_features)
                pred = trainer.qf(obs, obj_features, actions)
                print(f"debug {total_traj}/{total_step}", #"\n dist: ", dist.mean.flatten().tolist(), dist.stddev.flatten().tolist(),
                      "\n pred:", pred.flatten().tolist(),
                      "\n rewards: ", rewards.flatten().tolist())


    # Record
    record = env.scene.get_scene_data()
    env.scene.save_scene_data()
    # print("record: ", record)
    
    # Add record to buffer
    if use_network:
        buffer.add_scene_sample(record)

    # Reset (env clean)
    env.clean(clean_all = False)
    env.step(render = True)
    total_traj += 1

total_traj:  0
asset_prim: omniverse://localhost/Users/yizhou/Asset/T/Appleseed/Appleseed_CoffeeTable.usd
1.2] startup
[7.940s] [ext: omni.graph.window.core-1.23.4] startup
[7.948s] [ext: omni.graph.instancing-1.1.4] startup
[7.957s] [ext: omni.kit.menu.edit-1.0.6] startup
[7.960s] [ext: omni.isaac.isaac_sensor-1.0.2] startup
2023-01-22 20:37:08 [7,959ms] [Warning] [omni.physx.plugin] Deprecated: getSimulationEventStream is deprecated, please use getSimulationEventStreamV2
[7.976s] [ext: omni.graph.tutorials-1.1.2] startup
[7.997s] [ext: omni.graph.window.action-1.3.8] startup
[7.999s] [ext: omni.kit.widget.live-0.1.0] startup
[8.005s] [ext: omni.kit.widget.layers-1.5.17] startup
[8.022s] [ext: omni.graph.bundle.action-1.0.0] startup
[8.022s] [ext: omni.isaac.range_sensor-0.4.3] startup
[8.052s] [ext: omni.kit.property.layer-1.1.2] startup
[8.055s] [ext: omni.replicator.isaac-1.3.2] startup
[8.160s] [ext: omni.isaac.kit-0.2.1] startup
[8.161s] [ext: omni.graph.window.generic-1.3.8] sta

2023-01-22 20:37:55 [55,648ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Vase

2023-01-22 20:37:56 [55,874ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-22 20:37:56 [55,978ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-22 20:37:57 [57,101ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/GadgetsAndTool

2023-01-22 20:37:57 [57,102ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/GadgetsAndTool

2023-01-22 20:37:57 [57,102ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/GadgetsAndTool

2023-01-22 20:37:57 [57,102ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/GadgetsA

2023-01-22 20:37:59 [59,240ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/GadgetsAndTool_01

2023-01-22 20:37:59 [59,635ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-22 20:38:00 [59,739ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-22 20:38:01 [61,002ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware

2023-01-22 20:38:01 [61,002ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/GadgetsAndTool

2023-01-22 20:38:01 [61,002ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware

2023-01-22 20:38:01 [61,002ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Gad

2023-01-22 20:38:01 [61,532ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-22 20:38:01 [61,634ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-22 20:38:03 [62,853ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware

2023-01-22 20:38:03 [62,854ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/GadgetsAndTool

2023-01-22 20:38:03 [62,854ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware

2023-01-22 20:38:03 [62,854ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/GadgetsAndTool

2023-01-22 20:38:03 [62,854ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerwar

total_traj:  1
2023-01-22 20:38:03 [63,529ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-22 20:38:03 [63,631ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Vase/CellVase03.usd
2023-01-22 20:38:05 [64,864ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Vase

2023-01-22 20:38:05 [65,115ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-22 20:38:05 [65,219ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/GadgetsAndTool/red_grinder.usd
2023-01-22 20:38:06 [66,708ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /Worl

...

asset_prim: omniverse://localhost/Users/yizhou/Asset/I/GadgetsAndTool/coffee_grinder.usd
2023-01-22 20:58:54 [1,314,275ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/GadgetsAndTool_01

2023-01-22 20:58:55 [1,314,734ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-22 20:58:55 [1,314,837ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-22 20:58:56 [1,316,038ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Flatware

2023-01-22 20:58:56 [1,316,038ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Flatware

2023-01-22 20:58:56 [1,316,038ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Flatware

2023-01-22 20:58:56 [1,316,038ms] [Wa

debug 95/480 
 pred: [-1.0675649642944336, -0.9238578677177429, -0.9583752155303955, -1.037979245185852, -1.0289149284362793, -1.0243889093399048, -1.0592182874679565, -0.9630888104438782] 
 rewards:  [-0.4305419921875, -0.4306182861328125, -0.43062591552734375, -0.43062591552734375, -0.4305419921875, -0.43062591552734375, -0.43062591552734375, -1.0248757600784302]
total_traj:  96
2023-01-22 20:58:58 [1,318,608ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-22 20:58:58 [1,318,712ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Dinnerware/P_Glassware_Short.usd
2023-01-22 20:59:00 [1,319,859ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Dinnerware

2023-01-22 20:59:00 [1,320,044ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set ve

2023-01-22 20:59:07 [1,327,341ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Dinnerware_01

2023-01-22 20:59:07 [1,327,495ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-22 20:59:07 [1,327,597ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-22 20:59:09 [1,328,812ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware_01

2023-01-22 20:59:09 [1,328,813ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Flatware

2023-01-22 20:59:09 [1,328,813ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware_01

2023-01-22 20:59:09 [1,328,813ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /

total_traj:  97
2023-01-22 20:59:09 [1,329,500ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-22 20:59:09 [1,329,604ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Tchotchke/Orange_03.usd
2023-01-22 20:59:11 [1,330,797ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Tchotchke

2023-01-22 20:59:11 [1,331,132ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-22 20:59:11 [1,331,236ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Tchotchke/Orange_02.usd
2023-01-22 20:59:12 [1,332,642ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformO

2023-01-22 20:59:39 [1,359,088ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-22 20:59:39 [1,359,192ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Magazine/OpenMagazine.usd
2023-01-22 20:59:41 [1,360,860ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-22 20:59:41 [1,360,964ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
debug 99/500 
 pred: [-1.3024699687957764, -0.6640284657478333, -1.1623926162719727, -0.29245540499687195, -1.187854528427124, -1.3874942064285278, -1.1829534769058228, -0.9213857650756836] 
 rewards:  [-0.43062591552734375, 0.3314208984375, -0.43062591552734375, -0.43062591552734375, -0.43062591552734375, -0.43062591552734375, -0.43062591552734

# Debug

In [ ]:
env.clean(clean_base = True, clean_all = False)

In [ ]:
# get images
env.world.render()
env.world.render()
images = render.get_images()

Image.fromarray(images[0]['rgb'], "RGBA").resize((300, 300))

##  Debug Training

In [ ]:
import json
from learning.replay_buffer import ReplayBuffer

buffer = ReplayBuffer(max_size=1000)

for i in range(1):
    replay = json.load(open(f"{DATA_PATH}/{task_type}/{side_choice}/{i}/scene.json"))
    buffer.add_scene_sample(replay)

In [ ]:
len(buffer.dataset)

In [ ]:
model_type = "ddpg"

if model_type == "sac":
    # trainer
    from learning.network.sac import *

    policy = Policy()
    target_policy = Policy()

    qf1 = QFunction()
    qf2 = QFunction()

    target_qf1 = QFunction()
    target_qf1.eval()

    target_qf2 = QFunction()
    target_qf2.eval()

    from learning.sac_trainer import SACTrainer

    trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
         use_automatic_entropy_tuning = True, 
         policy_lr=1e-3, 
         qf_lr=1e-3,
         target_update_period = 5)

else: # ddpg
    # trainer
    from learning.network.ddpg import *

    policy = Policy()
    target_policy = Policy()

    qf1 = QFunction()
    target_qf1 = QFunction()
    target_qf1.eval()

    from learning.ddpg_trainer import DDPGTrainer
    
    trainer = DDPGTrainer(qf1, target_qf1, policy, target_policy,
     policy_learning_rate=1e-3,
     qf_learning_rate=1e-3
    )

In [ ]:
self = trainer

In [ ]:
# self.policy(obs, obj_features)

In [ ]:
for _ in range(100):
    batch = buffer.sample_batch(batch_size=4)

    rewards = batch['rewards'].to(device)
    terminals = batch['terminals'].to(device)
    obs = batch['observations'].to(device)
    actions = batch['actions'].to(device)
    next_obs = batch['next_observations'].to(device)

    obj_features = batch['object_features'].to(device)

    # print("rewards", rewards)

    self.update(batch)
    
    if _ % 10 == 0:
        policy = self.policy(obs, obj_features)
        pred = self.qf(obs, obj_features, actions)
        print(_, "\n policy   : ", [round(x, 3) for x in policy.flatten().tolist()], 
              "\n pred   :", [round(x, 3) for x in pred.flatten().tolist()],
              "\n rewards: ", [round(x, 3) for x in rewards.flatten().tolist()])
    
#     # debug
#     pred = self.qf1(obs, obj_features, actions)
#     loss = self.qf_criterion(pred, rewards)
    
#     self.qf1_optimizer.zero_grad()
#     loss.backward()
#     self.qf1_optimizer.step()
    
#     if _ % 20 == 0:
#         print(_, "\n debug loss: ", loss.item(), "\n pred:", 
#               pred.flatten().tolist(), "\n rewards: ", rewards.flatten().tolist())

In [ ]:
dist = self.policy(obs, obj_features)

In [ ]:
dist = self.policy(obs.to(device), obj_features.to(device))

In [ ]:
dist.normal_mean, dist.stddev

In [ ]:
dist.sample(scaler = 0.2)

In [ ]:
self = dist

In [ ]:
scaler = 0.1

In [ ]:
self.normal_mean + scaler * self.normal_std * MultivariateDiagonalNormal(
    torch.zeros(self.normal_mean.size()).to(self.device),
    torch.ones(self.normal_std.size()).to(self.device)
).sample()

# distribution

In [ ]:
from learning.distributions import MultivariateDiagonalNormal

In [ ]:
dist = JustNormal(dist.mean, dist.stddev)

In [ ]:
dist.rsample_and_logprob()